# 4.Advance Modeling<a id='4.Advance Modeling'></a>


## 4.1 Contents<a id='3.1_Contents'></a>

  * [4.2 Introduction](#4.2_Introduction)   
  * [4.3 Load The Data](#4.3_Load_The_Data)  
  * [4.4 Baseline model Using Linear regression](#4.4_Baseline_model_Using_Linear_regression)  
  * [4.5 KNN regression](#4.5_KNN_regression)      
  * [4.6 MAPE from both the models for the test set](#4.6_MAPE_from_both_the_models_for_the_test_set)
  * [4.7 Upper/Lower bound of the errors of residuals](#4.7_Upper/Lower_bound_of_the_errors_of_residualse)  
  * [4.8  Scale the Data ](#4.8_Scale_the_Data)
  * [4.9 Save data](#4.9_Save_data)
  * [4.10 Summary](#4.10_Summary)

## 4.2 Introduction<a id='4.2_Introduction'></a>

In [1]:

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import os
import numpy as np
%matplotlib inline
from matplotlib.gridspec import GridSpec
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
from sklearn.neighbors import KNeighborsRegressor
#!pip install xgboost

In [2]:
X_train = pd.read_pickle("./cleaned_data/X_tr_scaled.pkl") #X_tr_scaled.to_pickle("./cleaned_data/X_tr_scaled.pkl") 

In [3]:
X_test= pd.read_pickle("./cleaned_data/X_te_scaled.pkl") #X_te_scaled.to_pickle("./cleaned_data/X_te_scaled.pkl")

In [4]:
y_train= pd.read_pickle("./cleaned_data/y_train.pkl") #y_train.to_pickle("./cleaned_data/y_train.pkl")

In [5]:
y_test = pd.read_pickle("./cleaned_data/y_test.pkl") #y_test.to_pickle("./cleaned_data/y_test.pkl")

In [6]:
X_train.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.799560,0.181850,-0.817582,-0.539845,-0.757948,1.420006,0.155143,-0.285105,0.394301,0.494614,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855
1,0.321904,-0.314372,-1.550537,1.263166,-1.670795,-0.574380,-1.012563,-0.285105,0.108479,-1.091242,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855
2,-0.872237,-0.129258,0.648328,-0.539845,0.875568,-0.574380,-1.012563,-0.285105,2.236268,1.285008,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855
3,0.321904,1.552023,-0.084627,0.361661,-1.670795,-0.574380,1.621781,-0.285105,-0.753526,0.803678,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855
4,0.083076,-0.000721,0.648328,0.361661,1.067746,-0.574380,-1.012563,-0.285105,0.680123,-0.452846,...,-0.070117,3.257858,-0.044281,-2.469255,-0.275299,-0.054259,-0.062684,-0.117851,-2.148345,3.237769


In [7]:
X_test.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.872237,0.590728,0.648328,-0.539845,1.019701,-0.428052,-1.012563,-0.285105,2.485795,1.563673,...,-0.070117,3.257858,-0.044281,-2.469255,-0.275299,-0.054259,-0.062684,-0.117851,-2.148345,3.237769
1,-0.872237,-0.344885,0.648328,-0.539845,0.971657,-0.574380,-0.055044,-0.285105,0.537212,0.426214,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855
2,0.560732,-0.151635,-0.084627,-0.539845,-0.998171,0.829278,0.444734,1.843307,-0.649178,0.494614,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855
3,-0.872237,0.315218,1.381283,-0.539845,0.395122,-0.574380,1.862328,-0.285105,3.164056,5.439645,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855
4,2.471358,-0.959601,-0.817582,0.361661,-0.373591,-0.574380,0.267242,0.810774,-1.220822,-0.749244,...,-0.070117,-0.306950,-0.044281,0.404980,-0.275299,-0.054259,-0.062684,-0.117851,0.465475,-0.308855


In [8]:
y_train.head()

,SalePrice
1350,200000
1408,125500
276,201000
417,239000
686,227875


In [9]:
y_test.head()

,SalePrice
0,240000
1,185000
2,168500
3,430000
4,127000


## Random Forest Regressors

In [10]:
rf_model = RandomForestRegressor()

In [11]:
#converted the target  data into 1D
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [12]:
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [13]:
rf_predictions = rf_model.predict(X_test)

In [14]:
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_mape = mean_absolute_percentage_error(y_test, rf_predictions)

In [15]:
print('The mean absolute error with RandomForestRegressor is:', rf_mae, '\nThe mean absolute percentage error with RandomForestRegressor is:', rf_mape)

The mean absolute error with RandomForestRegressor is: 19684.070388127853 
The mean absolute percentage error with RandomForestRegressor is: 0.11772443771238916


## LGBM Regressors

In [16]:
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor()

In [17]:
lgb_model.fit(X_train, y_train)

LGBMRegressor()

In [18]:
lgb_predictions = lgb_model.predict(X_test)

In [19]:
lgb_mae = mean_absolute_error(y_test, lgb_predictions)
lgb_mape = mean_absolute_percentage_error(y_test, lgb_predictions)

In [20]:
print('The mean absolute error with LGBMRegressor is:', lgb_mae, '\nThe mean absolute percentage error with LGBMRegressor is:', lgb_mape )

The mean absolute error with LGBMRegressor is: 17976.290603201713 
The mean absolute percentage error with LGBMRegressor is: 0.1056428799172855


 ## XGBOOST Regressors

In [21]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor()

In [22]:
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [23]:
xgb_predictions = xgb_model.predict(X_test)

In [24]:
xgb_mae = mean_absolute_error(y_test, xgb_predictions)
xgb_mape = mean_absolute_percentage_error(y_test, xgb_predictions)

In [25]:
print('The mean absolute error with RandomForestRegressor is:', rf_mae, '\nThe mean absolute percentage error with RandomForestRegressor is:', rf_mape)

print('The mean absolute error with LGBMRegressor is:', lgb_mae, '\nThe mean absolute percentage error with LGBMRegressor is:', lgb_mape )

print('The mean absolute error with XGBRegressor is:', xgb_mae , '\nThe mean absolute percentage error with XGBRegressor is:', xgb_mape)

The mean absolute error with RandomForestRegressor is: 19684.070388127853 
The mean absolute percentage error with RandomForestRegressor is: 0.11772443771238916
The mean absolute error with LGBMRegressor is: 17976.290603201713 
The mean absolute percentage error with LGBMRegressor is: 0.1056428799172855
The mean absolute error with XGBRegressor is: 18707.06889447774 
The mean absolute percentage error with XGBRegressor is: 0.1118297795226431


## Hyperparameter Tuning using GridSearchCV 

#### Random Forest Regressors

In [26]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV

k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}


pipe = make_pipeline(
    SimpleImputer(strategy='median'), 
    StandardScaler(),
    SelectKBest(f_regression),
    LinearRegression()
)

k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}

lr_grid_cv = GridSearchCV(pipe, param_grid=grid_params, cv=5, n_jobs=-1)

rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

In [27]:
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

In [28]:
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

...

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\Users\asiya\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestR

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 5, 10],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

In [29]:
# Get the best Random Forest model from Grid Search
best_rf_model = rf_grid_search.best_estimator_
best_rf_model 

RandomForestRegressor(max_features='auto', n_estimators=200)

In [30]:
rf_grid_search.best_params_

{'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [31]:
y_pred = best_rf_model.predict(X_test)
tuned_rf_mae = mean_absolute_error(y_test, y_pred)
tuned_rf_mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error:", tuned_rf_mae)
print("Mean Absolute Percentage Error:", tuned_rf_mape)

Mean Squared Error: 19425.31506849315
Mean Absolute Percentage Error: 0.11691329317028115


In [32]:
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

In [33]:
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [3, 6, 9],
                         'max_features': ['sqrt', 'log2', None],
                         'max_leaf_nodes': [3, 6, 9],
                         'n_estimators': [25, 50, 100, 150]})

In [34]:
best_rf_model = rf_grid_search.best_estimator_
best_rf_model 

RandomForestRegressor(max_depth=9, max_features=None, max_leaf_nodes=9,
                      n_estimators=150)

In [35]:
print("Best parameters for Random Forest Regressor: ", rf_grid_search.best_params_)


Best parameters for Random Forest Regressor:  {'max_depth': 9, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}


In [36]:
y_pred = best_rf_model.predict(X_test)
tuned_rf_mae2 = mean_absolute_error(y_test, y_pred)
tuned_rf_mape2 = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error:", tuned_rf_mae2)
print("Mean Absolute Percentage Error:",tuned_rf_mape2)

Mean Squared Error: 25501.38526567061
Mean Absolute Percentage Error: 0.16544608564227314


#####  LGBM Regressors

In [37]:
k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}


# Perform Grid Search for LGBM
lgb_grid_search = GridSearchCV(estimator=lgb_model, param_grid=grid_params, cv=5, n_jobs=-1)
lgb_grid_search.fit(X_train, y_train)

# Get the best LGBM model from Grid Search
best_lgb_model = lgb_grid_search.best_estimator_
best_lgb_model 

[LightGBM] [Warning] Unknown parameter: selectkbest__k


LGBMRegressor(selectkbest__k=1)

In [38]:
print("Best parameters for LGBM Regressors: ", lgb_grid_search.best_params_)

Best parameters for LGBM Regressors:  {'selectkbest__k': 1}


In [39]:
y_pred = best_lgb_model.predict(X_test)
tuned_lgb_mae  = mean_absolute_error(y_test, y_pred)
tuned_lgb_mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error:", tuned_lgb_mae )
print("Mean Absolute Percentage Error:", tuned_lgb_mape)

Mean Squared Error: 17976.290603201713
Mean Absolute Percentage Error: 0.1056428799172855


#### XGBOOST Regressors

#Repeat the above steps for LGBM and XGBoost with their respective parameter grids

param_grid = {
    'n_estimators': [100, 200, 300],  # Number of gradient boosting iterations
    'learning_rate': [0.1, 0.01, 0.001],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of each tree
    'subsample': [0.8, 0.9],  # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2],  # Minimum loss reduction required to make a further partition on a leaf node
    'reg_alpha': [0, 0.01, 0.1],  # L1 regularization term on weights
    'reg_lambda': [0, 0.01, 0.1]  # L2 regularization term on weights
}

xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5,n_iter=10)
xgb_grid_search.fit(X_train, y_train)

best_xgb_model = xgb_grid_search.best_estimator_
best_xgb_model 

print("Best parameters for XGBOOST Regressors: ", xgb_grid_search.best_params_ )

y_pred = best_xgb_model .predict(X_test)
tuned_xgb_mae = mean_absolute_error(y_test, y_pred)
tuned_xgb_mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error:", tuned_xgb_mae)
print("Mean Absolute Percentage Error:", tuned_xgb_mape)

In [40]:
param_grid = {
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200],
    'max_depth': [3, 5],   
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)

# Get the best LGBM model from Grid Search
best_xgb_model = xgb_grid_search.best_estimator_
best_xgb_model 

print("Best parameters for XGBOOST Regressors: ", xgb_grid_search.best_params_)

y_pred = best_xgb_model.predict(X_test)
tuned_xgb_mae  = mean_absolute_error(y_test, y_pred)
tuned_xgb_mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error:", tuned_xgb_mae )
print("Mean Absolute Percentage Error:", tuned_xgb_mape)

Best parameters for XGBOOST Regressors:  {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Mean Squared Error: 16718.35353346176
Mean Absolute Percentage Error: 0.10031861494925615


###  comaprision of models after Hyperparameter Tuning using GridSearchCV 

In [41]:
print('The mean absolute error with RandomForestRegressor is:', rf_mae, '\nThe mean absolute percentage error with RandomForestRegressor is:', rf_mape)

print('The mean absolute error with LGBMRegressor is:', lgb_mae, '\nThe mean absolute percentage error with LGBMRegressor is:', lgb_mape )

print('The mean absolute error with XGBRegressor is:', xgb_mae , '\nThe mean absolute percentage error with XGBRegressor is:', xgb_mape)

The mean absolute error with RandomForestRegressor is: 19684.070388127853 
The mean absolute percentage error with RandomForestRegressor is: 0.11772443771238916
The mean absolute error with LGBMRegressor is: 17976.290603201713 
The mean absolute percentage error with LGBMRegressor is: 0.1056428799172855
The mean absolute error with XGBRegressor is: 18707.06889447774 
The mean absolute percentage error with XGBRegressor is: 0.1118297795226431


In [42]:
print('The mean absolute error with RandomForestRegressor after tuning:', tuned_rf_mae, '\nThe mean absolute percentage error with RandomForestRegressor after tuning is:', tuned_rf_mape)

print('The mean absolute error with LGBMRegressor after tuning is:', tuned_lgb_mae, '\nThe mean absolute percentage error with LGBMRegressor after tuning is:', tuned_lgb_mape )

print('The mean absolute error with XGBRegressor after tuning is:', tuned_xgb_mae , '\nThe mean absolute percentage error with XGBRegressor after tuning is:', tuned_xgb_mape)

The mean absolute error with RandomForestRegressor after tuning: 19425.31506849315 
The mean absolute percentage error with RandomForestRegressor after tuning is: 0.11691329317028115
The mean absolute error with LGBMRegressor after tuning is: 17976.290603201713 
The mean absolute percentage error with LGBMRegressor after tuning is: 0.1056428799172855
The mean absolute error with XGBRegressor after tuning is: 16718.35353346176 
The mean absolute percentage error with XGBRegressor after tuning is: 0.10031861494925615


In [43]:
print('The mean absolute error with RandomForestRegressor2 after tuning:', tuned_rf_mae2, '\nThe mean absolute percentage error with RandomForestRegressor2 after tuning is:', tuned_rf_mape2)

The mean absolute error with RandomForestRegressor2 after tuning: 25501.38526567061 
The mean absolute percentage error with RandomForestRegressor2 after tuning is: 0.16544608564227314
